# SCRIPT DE CÁLCULO DE MÉTRICAS Y ÁREA

Este script procesa las máscaras resultantes de la inferencia para:
 1. Calcular el Área superficial de la laguna (en metros cuadrados y km²).
 2. Calcular la métrica Dice Coefficient (si hay Ground Truth) usando TensorFlow.
 3. Guardar las imagenes segmentadas para validar los resultados


## Carga del modelo entrenado

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
        tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth
    )

# Carga un modelo previamente entrenado que utiliza el coeficiente Dice
# como métrica o función personalizada
model = tf.keras.models.load_model(
    'Modelo_Final.keras',
    custom_objects={'dice_coef': dice_coef}
)

## Imports y configuración inicial

In [ ]:
import os
import cv2
import numpy as np

# Tamaño de cada recorte (tile)
TILE_SIZE = 256                

# Ruta de la imagen a procesar
IMG_PATH = "PRUEBAS FINALES/2025_3rot.tif" 

# Carpeta donde se guardarán los recortes predichos
OUTPUT_DIR = "recortes_predichos" 
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Carga de la imagen

In [ ]:
# Leer imagen en formato BGR y convertirla a RGB
img = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Dimensiones de la imagen original
H, W, _ = img.shape
print("Tamaño original:", img.shape)


## División de la imagen en tiles

In [ ]:
tiles = []
positions = []

# Recorrer la imagen en bloques de 256x256
for y in range(0, H, TILE_SIZE):
    for x in range(0, W, TILE_SIZE):
        tile = img[y:y+TILE_SIZE, x:x+TILE_SIZE]

        # Rellenar con ceros si el tile es más pequeño (bordes)
        if tile.shape[0] != TILE_SIZE or tile.shape[1] != TILE_SIZE:
            padded = np.zeros((TILE_SIZE, TILE_SIZE, 3), dtype=np.uint8)
            padded[:tile.shape[0], :tile.shape[1]] = tile
            tile = padded

        tiles.append(tile)
        positions.append((y, x))

# Normalizar para el modelo
tiles_np = np.array(tiles) / 255.0
print(f"Total de recortes: {len(tiles_np)}")

## Predicción con el modelo

In [ ]:
# Predicción de máscaras por tile
pred_masks = model.predict(tiles_np, verbose=1)

# Umbralización: probabilidades → máscara binaria
pred_masks = (pred_masks > 0.05).astype(np.uint8)

## Reconstrucción de la máscara completa

In [ ]:
# Máscara final del tamaño de la imagen original
full_mask = np.zeros((H, W), dtype=np.uint8)

tile_idx = 0
for (y, x) in positions:
    # Escalar máscara a valores 0-255
    mask_tile = pred_masks[tile_idx, :, :, 0] * 255
    
    # Ajustar tamaño en bordes
    h = min(TILE_SIZE, H - y)
    w = min(TILE_SIZE, W - x)

    full_mask[y:y+h, x:x+w] = mask_tile[:h, :w]

    # Guardar cada recorte predicho (opcional)
    out_path = os.path.join(OUTPUT_DIR, f"mask_tile_{tile_idx}.png")
    cv2.imwrite(out_path, mask_tile)

    tile_idx += 1

## Cálculo de área

In [ ]:
# Conteo de píxeles clasificados como lago
pixels_laguna = np.sum(full_mask == 255)
print("Pixeles de lago:", pixels_laguna)

# Resolución espacial (metros por pixel)
resolution_m = 4.7  # ajustar según el sensor
area_m2 = pixels_laguna * (resolution_m**2)

print(f"Área aproximada: {area_m2} m²")

## Visualización de los resultados

In [ ]:
import matplotlib.pyplot as plt

output_dir = "Imagenes"
os.makedirs(output_dir, exist_ok=True)

plt.figure(figsize=(12, 12))

# Imagen original
plt.subplot(1, 2, 1)
plt.title("Imagen")
plt.imshow(img)
plt.axis('off')

# Máscara predicha
plt.subplot(1, 2, 2)
plt.title("Predicción")
plt.imshow(full_mask, cmap='gray')
plt.axis('off')

plt.tight_layout()

# Guardar figura (opcional)
# plt.savefig(os.path.join(output_dir, "2025_4.png"), dpi=300)

plt.show()